In [1]:
import os 
import pickle
import numpy as np
from os.path import join  

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.distribute import MirroredStrategy, OneDeviceStrategy

from utils import *

datasets = join(os.getcwd(), "datasets")
preprocessed_datasets = join(datasets, "preprocessed")

X_all_sep, y_all, X_test_sep = pickle.load(open(join(preprocessed_datasets, "cleaned_separated.pickle"), 'rb'))
X_all_ohe, _, X_test_ohe = pickle.load(open(join(preprocessed_datasets, "cleaned_ohe.pickle"), 'rb'))

ohe_split =  X_all_ohe.shape[1] - X_all_sep[0].shape[1]
X_all = (*X_all_sep, X_all_ohe[:,-ohe_split:])
X_test = (*X_test_sep, X_test_ohe[:,-ohe_split:])

strategy = OneDeviceStrategy("CPU")

In [2]:
def train_test_tuple_split(X, y, train_size, seed=None):
    def apply_tuple_split(X, split):
        train, test = [], []
        for x in X:
            train.append(x[:split])
            test.append(x[split:])
        return tuple(train), tuple(test)
    
    np.random.seed(seed)
    
    indices_ = np.arange(len(y))
    np.random.shuffle(indices_)
    
    splitter_indice = int(len(y) * train_size)
    train_indices, test_indices = np.split(indices_, [splitter_indice])
    
    X_train, X_test = apply_tuple_split(X, splitter_indice)
    y_train, y_test = y[train_indices], y[test_indices]
    
    return X_train, X_test, y_train, y_test
    

X_train, X_val, y_train, y_val = train_test_tuple_split(X_all, y_all, .75, seed=42)

In [3]:
with strategy.scope():
    
    conv_len, vp_len, ohe_len = len(X_train[0][-1]), len(X_train[1][-1]), len(X_train[2][-1])
    # vocab_dim = int(1.2*len(np.unique(np.r_[X_train[1], X_test[1]])))
    vocab_dim = int(np.unique(np.r_[X_train[1], X_test[1]]).max())+1
    
    conv_input = tf.keras.layers.Input(shape=(conv_len,))
    
    
    vp_input = tf.keras.layers.Input(shape=(vp_len,))
    embedded = tf.keras.layers.Embedding(vocab_dim, 128, input_length=vp_len)(vp_input)
    dense_emb = tf.keras.layers.Dense(1)(embedded)
    flatten_emb = tf.keras.layers.Flatten()(dense_emb)
    
    ohe_input = tf.keras.layers.Input(shape=(ohe_len,))
    
    concat = tf.keras.layers.Concatenate()([conv_input, flatten_emb, ohe_input])
    
    output = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.L1(0.01), activation='sigmoid')(concat)
    
    model = tf.keras.models.Model(inputs=[conv_input, vp_input, ohe_input], outputs=output)
    
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(lr=1e-3),
        metrics=[bcr, p1, p2, m1, m2, "accuracy"]
    )

callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_bcr', factor=.5, patience=10, verbose=2),
    BCREarlyStopping(patience=30, restore_best_weights=True)
]

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 6, 128)       4480        input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 6, 1)         129         embedding[0][0]                  
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1306624)]    0                                            
______________________________________________________________________________________________

In [4]:
history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val),
                    batch_size=120,
                    epochs=200,
                    callbacks=callbacks
                   )

Epoch 1/200
2/2 [==============================] - ETA: 2s - loss: 24.5676 - bcr: 0.5059 - p1: 1.0000 - p2: 0.0118 - m1: 35.0000 - m2: 85.0000 - accuracy: 0.300 - ETA: 0s - loss: 605.7090 - bcr: 0.5044 - p1: 0.7500 - p2: 0.2588 - m1: 36.2500 - m2: 81.5000 - accuracy: 0.38 - 4s 2s/step - loss: 799.4228 - bcr: 0.5039 - p1: 0.6667 - p2: 0.3412 - m1: 36.6667 - m2: 80.3333 - accuracy: 0.4088 - val_loss: 4484.6787 - val_bcr: 0.5000 - val_p1: 0.0000e+00 - val_p2: 1.0000 - val_m1: 41.0000 - val_m2: 37.0000 - val_accuracy: 0.4744


/home/romaingrx/Nextcloud/EPL/Q8/Cours/LINGI2262 - Machine Learning/LINGI2262-Projects/Assignment 5 - A Machine Learning Competition/utils.py:105: RuntimeWarning: divide by zero encountered in double_scalars
  return bcr - bcr_delta * (1 - np.exp(-bcr_delta / sigma))


New best p value : 0.5
Epoch 2/200
2/2 [==============================] - ETA: 0s - loss: 3051.6477 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 43.0000 - m2: 77.0000 - accuracy: 0.641 - ETA: 0s - loss: 2868.6289 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 40.2500 - m2: 77.5000 - accuracy: 0.658 - 1s 428ms/step - loss: 2807.6226 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 39.3333 - m2: 77.6667 - accuracy: 0.6641 - val_loss: 3273.9280 - val_bcr: 0.5000 - val_p1: 0.0000e+00 - val_p2: 1.0000 - val_m1: 41.0000 - val_m2: 37.0000 - val_accuracy: 0.4744


/home/romaingrx/Nextcloud/EPL/Q8/Cours/LINGI2262 - Machine Learning/LINGI2262-Projects/Assignment 5 - A Machine Learning Competition/utils.py:105: RuntimeWarning: invalid value encountered in double_scalars
  return bcr - bcr_delta * (1 - np.exp(-bcr_delta / sigma))


Epoch 3/200
2/2 [==============================] - ETA: 0s - loss: 1714.6387 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 33.0000 - m2: 87.0000 - accuracy: 0.725 - ETA: 0s - loss: 1641.3342 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 35.2500 - m2: 82.5000 - accuracy: 0.700 - 1s 430ms/step - loss: 1616.8994 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 36.0000 - m2: 81.0000 - accuracy: 0.6919 - val_loss: 253.2992 - val_bcr: 0.5000 - val_p1: 0.0000e+00 - val_p2: 1.0000 - val_m1: 41.0000 - val_m2: 37.0000 - val_accuracy: 0.4744
Epoch 4/200
2/2 [==============================] - ETA: 0s - loss: 141.4025 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 37.0000 - m2: 83.0000 - accuracy: 0.69 - ETA: 0s - loss: 618.0826 - bcr: 0.5000 - p1: 0.2500 - p2: 0.7500 - m1: 37.2500 - m2: 80.5000 - accuracy: 0.6077   - 1s 438ms/step - loss: 776.9760 - bcr: 0.5000 - p1: 0.3333 - p2: 0.6667 - m1: 37.3333 - m2: 79.6667 - accuracy: 0.5798 - val_loss: 2037.4734 - val_bcr: 0.5000 - val_p1: 1.0

Epoch 18/200
2/2 [==============================] - ETA: 0s - loss: 177.4795 - bcr: 0.5122 - p1: 0.0244 - p2: 1.0000 - m1: 41.0000 - m2: 79.0000 - accuracy: 0.66 - ETA: 0s - loss: 150.8767 - bcr: 0.5410 - p1: 0.0918 - p2: 0.9903 - m1: 39.2500 - m2: 78.5000 - accuracy: 0.68 - 1s 450ms/step - loss: 142.0091 - bcr: 0.5507 - p1: 0.1143 - p2: 0.9870 - m1: 38.6667 - m2: 78.3333 - accuracy: 0.6955 - val_loss: 109.6397 - val_bcr: 0.5837 - val_p1: 0.9512 - val_p2: 0.2162 - val_m1: 41.0000 - val_m2: 37.0000 - val_accuracy: 0.6026
Epoch 19/200
2/2 [==============================] - ETA: 0s - loss: 76.3415 - bcr: 0.7099 - p1: 1.0000 - p2: 0.4198 - m1: 39.0000 - m2: 81.0000 - accuracy: 0.608 - ETA: 0s - loss: 93.6233 - bcr: 0.6724 - p1: 1.0000 - p2: 0.3448 - m1: 38.2500 - m2: 79.5000 - accuracy: 0.559 - 1s 443ms/step - loss: 99.3840 - bcr: 0.6599 - p1: 1.0000 - p2: 0.3198 - m1: 38.0000 - m2: 79.0000 - accuracy: 0.5433 - val_loss: 75.0641 - val_bcr: 0.4654 - val_p1: 0.3902 - val_p2: 0.5405 - val_m1:

2/2 [==============================] - ETA: 0s - loss: 18.2623 - bcr: 0.9080 - p1: 0.9070 - p2: 0.9091 - m1: 43.0000 - m2: 77.0000 - accuracy: 0.908 - ETA: 0s - loss: 18.7904 - bcr: 0.8849 - p1: 0.8443 - p2: 0.9255 - m1: 40.2500 - m2: 77.5000 - accuracy: 0.902 - 1s 456ms/step - loss: 18.9665 - bcr: 0.8772 - p1: 0.8234 - p2: 0.9310 - m1: 39.3333 - m2: 77.6667 - accuracy: 0.9002 - val_loss: 128.5657 - val_bcr: 0.4812 - val_p1: 0.0976 - val_p2: 0.8649 - val_m1: 41.0000 - val_m2: 37.0000 - val_accuracy: 0.4615
Epoch 34/200
2/2 [==============================] - ETA: 0s - loss: 25.6593 - bcr: 0.7632 - p1: 0.5263 - p2: 1.0000 - m1: 38.0000 - m2: 82.0000 - accuracy: 0.850 - ETA: 0s - loss: 25.8964 - bcr: 0.7616 - p1: 0.5299 - p2: 0.9932 - m1: 37.7500 - m2: 80.0000 - accuracy: 0.844 - 1s 465ms/step - loss: 25.9755 - bcr: 0.7610 - p1: 0.5311 - p2: 0.9910 - m1: 37.6667 - m2: 79.3333 - accuracy: 0.8432 - val_loss: 89.6037 - val_bcr: 0.4908 - val_p1: 0.1707 - val_p2: 0.8108 - val_m1: 41.0000 - val

In [5]:
report = Report(model, X_train, y_train, X_val, y_val).to_stdout()

------------------ Report for Functional -------------------

P score : 0.573
BCR     : 0.673
BCR hat : 0.569
